In [93]:
import math
import numpy as np
import pandas as pd

In [94]:
# Training and test data files
train_file = '../input/train.csv'
test_file = '../input/test.csv'
model_file = '../output/titanic.model.json'
model_weights_file = '../output/titanic.model.best.hdf5'
pred_file = '../output/gender_submission.csv'

In [95]:
# Load training data
df_train = pd.read_csv(train_file)
print(df_train.shape)
df_train.info()
df_train.head()

(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [96]:
# Extract survived data as predictions: 0 = Died, 1 = Survived
from keras.utils.np_utils import to_categorical
y_train = to_categorical(df_train["Survived"], 2)
df_train.pop('Survived')
print(y_train.shape)
print(y_train[0:5])

(891, 2)
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [97]:
# Prepare the rest of the data for training
from sklearn.preprocessing import MinMaxScaler

max_name_len = df_train.Name.map(len).max()
max_ticket_len = df_train.Ticket.map(len).max()

title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev', 'Dr', 'Ms', 'Mlle',
            'Col', 'Capt', 'Mme', 'Countess', 'Don', 'Jonkheer']

import string
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if substring in big_string:
            return substring
    return np.nan

def prep_data(frame):
    # Fill missing Age data with median 
    frame['Age'] = frame['Age'].fillna(frame['Age'].median())
    
    # Generate data about whether adult or minor
    frame['Adult_Or_Minor'] = frame.apply(lambda row: 0 if row['Age'] < 18 else 1, axis=1)

    # Fill missing Fare data with median
    frame['Fare'] = frame['Fare'].fillna(frame['Fare'].median())
    
    # Creating new family_size and fare per person columns 
    frame['Family_Size'] = frame['SibSp'] + frame['Parch'] + 1
    frame['Fare_Per_Person'] = frame['Fare']/frame['Family_Size']

    # Convert Sex and Embarked to number
    frame['Sex'] = pd.Categorical(frame['Sex']).codes
    frame['Age'] = frame['Age'].fillna('S')
    frame['Embarked'] = pd.Categorical(frame['Embarked']).codes
    
    # Extract title from name
    frame['Title'] = frame['Name'].map(lambda x: substrings_in_string(x, title_list))
    frame['Title'] = pd.Categorical(frame['Title']).codes

    # Convert Name into characters
    frame['Name_Length'] = frame.apply(lambda row: len(row['Name']), axis=1)
    for i in range(0, max_name_len):
        col_name = 'Name' + str(i)
        frame[col_name] = frame['Name'].str[i]
        frame[col_name] = frame.apply(lambda row: 0 if isinstance(row[col_name], float) and math.isnan(row[col_name]) else ord(row[col_name]), axis=1)
    frame.pop('Name')    
    
    # Convert Ticket into characters
    frame['Ticket_Length'] = frame.apply(lambda row: len(row['Ticket']), axis=1)
    for i in range(0, max_ticket_len):
        col_name = 'Ticket' + str(i)
        frame[col_name] = frame['Ticket'].str[i]
        frame[col_name] = frame.apply(lambda row: 0 if isinstance(row[col_name], float) and math.isnan(row[col_name]) else ord(row[col_name]), axis=1)
    frame.pop('Ticket')    
    
    # Convert Cabin column to whether in cabin
    frame['Cabin'] = frame['Age'].fillna(0)
    frame['In_Cabin'] = frame.apply(lambda row: 1 if row['Cabin'] != 0 else 0, axis=1)
    frame.pop('Cabin')
    
    frame.fillna(0, axis=1)
    
    print("Before scaling: ")
    print(frame.head())
    
    # Scale everything except PassengerId
    min_max_scaler = MinMaxScaler()
    col_list = frame.columns.tolist()
    col_list.remove('PassengerId')
    frame = frame[col_list]
    np_scaled = min_max_scaler.fit_transform(frame)
    frame = pd.DataFrame(np_scaled)
    
    print("After scaling: ")
    print(frame.head())

    return frame


In [98]:
# Prepare the rest of the data for training
df_train = prep_data(df_train)
X_train = np.array(df_train)[:,:]
X_train = X_train.astype('float32')
print(X_train.shape)
print(X_train[0])

Before scaling: 
   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  \
0            1       3    1  22.0      1      0   7.2500         2   
1            2       1    0  38.0      1      0  71.2833         0   
2            3       3    0  26.0      0      0   7.9250         2   
3            4       1    0  35.0      1      0  53.1000         2   
4            5       3    1  35.0      0      0   8.0500         2   

   Adult_Or_Minor  Family_Size    ...     Ticket9  Ticket10  Ticket11  \
0               1            2    ...           0         0         0   
1               1            2    ...           0         0         0   
2               1            1    ...          51        49        48   
3               1            2    ...           0         0         0   
4               1            1    ...           0         0         0   

   Ticket12  Ticket13  Ticket14  Ticket15  Ticket16  Ticket17  In_Cabin  
0         0         0         0         0        

In [107]:
# Build a training network

from keras.models import Sequential
from keras.layers import Dense, Dropout, RepeatVector, Flatten, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()
model.add(Dense(891, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(445, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 891)               102465    
_________________________________________________________________
dropout_35 (Dropout)         (None, 891)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 445)               396940    
_________________________________________________________________
dropout_36 (Dropout)         (None, 445)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 2)                 892       
Total params: 500,297
Trainable params: 500,297
Non-trainable params: 0
_________________________________________________________________


In [108]:
# Save the model
model_json = model.to_json()
with open(model_file, 'w') as json_file:
    json_file.write(model_json)

In [109]:
# Train the model
checkpointer = ModelCheckpoint(filepath=model_weights_file, verbose=1, save_best_only=True)
stopper = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1, mode='auto')
hist = model.fit(X_train, y_train, epochs=1000, batch_size=15, validation_split=0.2,
                 callbacks=[checkpointer, stopper], 
                 verbose=1, shuffle=True)

Train on 712 samples, validate on 179 samples
Epoch 1/1000
712/712 [==============================] - 1s - loss: 0.6868 - acc: 0.5281 - val_loss: 0.6453 - val_acc: 0.6425
Epoch 2/1000
712/712 [==============================] - 0s - loss: 0.6479 - acc: 0.6250 - val_loss: 0.6111 - val_acc: 0.6425
Epoch 3/1000
712/712 [==============================] - 0s - loss: 0.6290 - acc: 0.6334 - val_loss: 0.5873 - val_acc: 0.7207
Epoch 4/1000
712/712 [==============================] - 0s - loss: 0.5947 - acc: 0.6868 - val_loss: 0.5634 - val_acc: 0.7598
Epoch 5/1000
712/712 [==============================] - 0s - loss: 0.5763 - acc: 0.7051 - val_loss: 0.5388 - val_acc: 0.7765
Epoch 6/1000
712/712 [==============================] - 0s - loss: 0.5575 - acc: 0.7107 - val_loss: 0.5195 - val_acc: 0.7821
Epoch 7/1000
712/712 [==============================] - 0s - loss: 0.5529 - acc: 0.7219 - val_loss: 0.5097 - val_acc: 0.7598
Epoch 8/1000
712/712 [==============================] - 0s - loss: 0.5330 - acc

712/712 [==============================] - 0s - loss: 0.4307 - acc: 0.8104 - val_loss: 0.4129 - val_acc: 0.8156
Epoch 29/1000
712/712 [==============================] - 0s - loss: 0.4388 - acc: 0.8034 - val_loss: 0.4229 - val_acc: 0.8268
Epoch 30/1000
712/712 [==============================] - 0s - loss: 0.4272 - acc: 0.8104 - val_loss: 0.4128 - val_acc: 0.8101
Epoch 31/1000
712/712 [==============================] - 0s - loss: 0.4368 - acc: 0.8132 - val_loss: 0.4164 - val_acc: 0.7989
Epoch 32/1000
712/712 [==============================] - 0s - loss: 0.4331 - acc: 0.8146 - val_loss: 0.4107 - val_acc: 0.8101
Epoch 33/1000
712/712 [==============================] - 0s - loss: 0.4350 - acc: 0.8118 - val_loss: 0.4097 - val_acc: 0.8101
Epoch 34/1000
712/712 [==============================] - 0s - loss: 0.4285 - acc: 0.8146 - val_loss: 0.4091 - val_acc: 0.8101
Epoch 35/1000
712/712 [==============================] - 0s - loss: 0.4311 - acc: 0.8188 - val_loss: 0.4087 - val_acc: 0.8101
Epoch 

712/712 [==============================] - 0s - loss: 0.4021 - acc: 0.8371 - val_loss: 0.3997 - val_acc: 0.8156
Epoch 60/1000
712/712 [==============================] - 0s - loss: 0.3866 - acc: 0.8385 - val_loss: 0.4032 - val_acc: 0.8268
Epoch 61/1000
712/712 [==============================] - 0s - loss: 0.3858 - acc: 0.8301 - val_loss: 0.3990 - val_acc: 0.8212
Epoch 62/1000
712/712 [==============================] - 0s - loss: 0.4020 - acc: 0.8329 - val_loss: 0.3988 - val_acc: 0.8212
Epoch 63/1000
712/712 [==============================] - 0s - loss: 0.4005 - acc: 0.8230 - val_loss: 0.4001 - val_acc: 0.8324
Epoch 64/1000
712/712 [==============================] - 0s - loss: 0.3915 - acc: 0.8385 - val_loss: 0.4009 - val_acc: 0.8268
Epoch 65/1000
712/712 [==============================] - 0s - loss: 0.3895 - acc: 0.8301 - val_loss: 0.4065 - val_acc: 0.8324
Epoch 66/1000
712/712 [==============================] - 0s - loss: 0.3931 - acc: 0.8315 - val_loss: 0.3989 - val_acc: 0.8212
Epoch 

In [110]:
# Load the weights that yielded the best validation accuracy
model.load_weights(model_weights_file)

# Evaluate the model on the training set
score = model.evaluate(X_train, y_train)
print("\nTraining Accuracy:", score[1])

672/891 [=====================>........] - ETA: 0s
Training Accuracy: 0.84960718381


In [111]:
# Load test data
df_test_raw = pd.read_csv(test_file)
print(df_test_raw.shape)
df_test_raw.head()
df_test_raw.info()

(418, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [112]:
# Prepare the data for testing
df_test = prep_data(df_test_raw)
X_test = np.array(df_test)[:,:]
X_test = X_test.astype('float32')
print(X_test.shape)
print(X_test[0])

Before scaling: 
   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  \
0          892       3    1  34.5      0      0   7.8292         1   
1          893       3    0  47.0      1      0   7.0000         2   
2          894       2    1  62.0      0      0   9.6875         1   
3          895       3    1  27.0      0      0   8.6625         2   
4          896       3    0  22.0      1      1  12.2875         2   

   Adult_Or_Minor  Family_Size    ...     Ticket9  Ticket10  Ticket11  \
0               1            1    ...           0         0         0   
1               1            2    ...           0         0         0   
2               1            1    ...           0         0         0   
3               1            1    ...           0         0         0   
4               1            3    ...           0         0         0   

   Ticket12  Ticket13  Ticket14  Ticket15  Ticket16  Ticket17  In_Cabin  
0         0         0         0         0        

In [113]:
# Predict for test data
y_test = model.predict(X_test)
print(y_test[0])

[ 0.34917012  0.04882123]


In [114]:
# Save predictions
with open(pred_file, 'w') as f:
    f.write('PassengerId,Survived\n')
    for index, y_hat in enumerate(y_test):
        prediction = np.argmax(y_hat)
        f.write(str(int(df_test_raw.iloc[index]['PassengerId'])) + ',' + str(prediction)+'\n')
    f.close()